## Import all Libraries

In [1]:
# AWS Rekognition to get bbox
import numpy as np
import boto3
from PIL import Image, ImageDraw, ExifTags, ImageColor, ImageFont
from matplotlib import pyplot as plt
from utils.rekognition import determine_color, draw_animal_count
import time
import math
import os
import glob
import io
import json
FNT = ImageFont.truetype('/usr/share/fonts/default/Type1/n019004l.pfb', 25)

## Get Bounding Boxes from Video Frames

In [2]:
#draw response
def draw_response(image, response, animal_target):
    # original image size
    draw = ImageDraw.Draw(image)

    for customLabel in response['CustomLabels']:

        if 'Geometry' in customLabel:
            box = customLabel['Geometry']['BoundingBox']
            left = box['Left']
            top = box['Top']
            width = box['Width']
            height = box['Height']
            #print(customLabel['Name'])
            label = customLabel['Name']
        
            text = label
            text_width, text_height = FNT.getsize(label)
            color = determine_color(label, True)

            button_width = int(text_width + 20)
            button_height = int(text_height + 15)
            button_size = (button_width, button_height)
            button_img = Image.new('RGB', button_size, color)
            button_draw = ImageDraw.Draw(button_img)
            button_draw.text((10, 10), text, fill ='#000000', font=FNT)
            image.paste(button_img, (int(left), int(top)))  

            points = (
                (left, top),
                (left + width, top),
                (left + width, top + height),
                (left , top + height),
                (left, top))
            
            thickness = 5
            
            if label == 'cow':
                thickness = 7
                
            draw.line(points, fill=color, width=thickness)
    
    img = np.asarray(image)[:,:,::-1].copy()
    return img

In [3]:
def get_bbox_json(src_file, output_file, projectVersionArn, min_confidence=75):
    start = time.time()
    rekognition = boto3.client('rekognition')
    count = 0
    # get total frames
    frame_total = 0
    for infile in os.listdir(src_file):
        if infile.endswith(".jpg"):
            frame_total += 1
    print('Total frames:', frame_total)
    frame_res = [{} for _ in range(frame_total)]
    # process each image file
    for infile in os.listdir(src_file):
        if infile.endswith(".jpg"):
            file, ext = os.path.splitext(infile)
            frameId = int(file)
            #print('FrameId', frameId)
            with Image.open(src_file+file+ext) as img:
                #print(img.size)
                img_byte_arr = io.BytesIO()
                img.save(img_byte_arr, format='PNG')
                img_byte_arr = img_byte_arr.getvalue()
                # get labels
                response = rekognition.detect_custom_labels(
                    Image={
                        'Bytes': img_byte_arr,
                    },
                    ProjectVersionArn=projectVersionArn
                )
                imgWidth, imgHeight = img.size
                # scale bbox coordiantion to image size 
                for idx, customLabel in enumerate(response['CustomLabels']):
                    if 'Geometry' in customLabel:
                        box = customLabel['Geometry']['BoundingBox']
                        box['Left'] = imgWidth * box['Left']
                        box['Top'] = imgHeight * box['Top']
                        box['Width'] = imgWidth * box['Width']
                        box['Height'] = imgHeight * box['Height']
#                 if frameId == 200:
#                     inferred_frame = draw_response(img, response, animal_target='cow')
#                     plt.imshow(inferred_frame)
#                     plt.title("Frame {}".format(int(frameId)))
#                     plt.show()
                frame_res[frameId] = {'CustomLabels': response['CustomLabels']}
                    
            count+=1
            if count % 10 == 0:
                print("Finish Processing {} frame".format(count))
                lap = time.time()
                print('lap time: ', lap - start)
                
    #print(frame_res[617])
    #the json file where the output must be stored
    with open(output_file, 'w+') as out_file:
        json.dump({'Frames':frame_res}, out_file)
        out_file.close()
    end = time.time()
    print('total time lapse', end - start)

In [4]:
six_class_arn = 'arn:aws:rekognition:us-east-1:617989056061:project/horses-level_1/version/horses-level_1.2021-06-02T20.38.06/1622691487252'
# 'Maternity Pen 1_00_20210420044929', 'Maternity Pen 1_00_20210420044420', 'Maternity Pen 1_00_20210420044503'
#, 'cattle_multi_1', 'cattle_single_1' 
video_name_list = ['IMG_1501', 'IMG_1511', 'Sigar']
for video_name in video_name_list:
    src_path = 'frame_img/'+video_name+'/'
    output_json_path = 'json_data/'+video_name+'_horse_bbox.json'
    print(src_path)
    print(output_json_path)
    get_bbox_json(src_path, output_json_path, six_class_arn)
print('finished estimating bbox.')

frame_img/IMG_1501/
json_data/IMG_1501_horse_bbox.json
Total frames: 423
Finish Processing 10 frame
lap time:  34.1775758266449
Finish Processing 20 frame
lap time:  67.44304609298706
Finish Processing 30 frame
lap time:  100.5853853225708
Finish Processing 40 frame
lap time:  133.89092540740967
Finish Processing 50 frame
lap time:  167.0763783454895
Finish Processing 60 frame
lap time:  200.35980606079102
Finish Processing 70 frame
lap time:  234.8422405719757
Finish Processing 80 frame
lap time:  268.0737979412079
Finish Processing 90 frame
lap time:  301.22893619537354
Finish Processing 100 frame
lap time:  334.730996131897
Finish Processing 110 frame
lap time:  368.36939811706543
Finish Processing 120 frame
lap time:  401.8561022281647
Finish Processing 130 frame
lap time:  434.9080047607422
Finish Processing 140 frame
lap time:  468.37588334083557
Finish Processing 150 frame
lap time:  501.49266386032104
Finish Processing 160 frame
lap time:  534.8541970252991
Finish Processing 17

Finish Processing 250 frame
lap time:  842.7985563278198
Finish Processing 260 frame
lap time:  877.305433511734
Finish Processing 270 frame
lap time:  910.7170732021332
Finish Processing 280 frame
lap time:  944.098226070404
Finish Processing 290 frame
lap time:  978.1258494853973
Finish Processing 300 frame
lap time:  1011.9472484588623
Finish Processing 310 frame
lap time:  1045.5593118667603
Finish Processing 320 frame
lap time:  1079.5760588645935
Finish Processing 330 frame
lap time:  1112.8200242519379
Finish Processing 340 frame
lap time:  1146.0865473747253
Finish Processing 350 frame
lap time:  1179.8691132068634
Finish Processing 360 frame
lap time:  1213.586404800415
Finish Processing 370 frame
lap time:  1247.4319531917572
Finish Processing 380 frame
lap time:  1281.1859192848206
Finish Processing 390 frame
lap time:  1315.01522397995
Finish Processing 400 frame
lap time:  1348.667180776596
Finish Processing 410 frame
lap time:  1382.1749863624573
Finish Processing 420 fra

Finish Processing 1670 frame
lap time:  5651.696219444275
Finish Processing 1680 frame
lap time:  5684.587481498718
Finish Processing 1690 frame
lap time:  5717.867166757584
Finish Processing 1700 frame
lap time:  5751.374537229538
Finish Processing 1710 frame
lap time:  5784.5816531181335
Finish Processing 1720 frame
lap time:  5819.015102148056
Finish Processing 1730 frame
lap time:  5853.548288106918
Finish Processing 1740 frame
lap time:  5887.269040584564
Finish Processing 1750 frame
lap time:  5921.686683416367
Finish Processing 1760 frame
lap time:  5955.580899238586
Finish Processing 1770 frame
lap time:  5990.617305755615
Finish Processing 1780 frame
lap time:  6024.3322060108185
Finish Processing 1790 frame
lap time:  6057.987433195114
Finish Processing 1800 frame
lap time:  6091.663161039352
Finish Processing 1810 frame
lap time:  6125.7479746341705
Finish Processing 1820 frame
lap time:  6159.846149921417
Finish Processing 1830 frame
lap time:  6193.159125804901
Finish Proc